## **Prediction Rule Ensembles**

From the paper **Fitting Prediction Rule Ensembles with R Package pre** located in https://arxiv.org/pdf/1707.07149.pdf

In [ ]:
## **Example: prediction of depression**

Predict five major personality dimensions: Neuroticism, Extraversion, Openness to Experience, Agreeableness and Conscientiousness.